In [1]:
import pandas as pd
import gzip

In [2]:
def Create_taxID_to_SciName(Path="Data/sci_names.dmp.gz"):
    with gzip.open(Path, "rt") as file:
        Name_Lines = [Entry.split("\t") for Entry in file.readlines()]

    taxID_to_SciName = {}
    for Entry in Name_Lines:
        taxID_to_SciName[Entry[0]] = Entry[3]
    
    return taxID_to_SciName

taxID_to_SciName = Create_taxID_to_SciName(Path="Data/sci_names.dmp.gz")

In [3]:
# def taxID_to_SciName(Path="Data/sci_names.dmp.gz"):
#     ### Get files
with gzip.open("Data/GenBank_Bacterial_taxID_lineage.tsv.gz", "rt") as file:
    GB_Lines = [Entry.split("\t") for Entry in file.readlines()]

with gzip.open("Data/IBM_assemblies_taxonomy_metadata.tsv.gz", "rt") as file:
    IBM_Lines = [Entry.split("\t") for Entry in file.readlines()]

In [4]:
Acc_to_Lineage = {}
Acc_to_Rank = {}

### GenBank
for Line in GB_Lines[1:]:

    Accession = Line[0].split(".")[0]

    taxID = Line[1]

    taxID_lineage = Line[2].split(",")
    Rank_lineage = Line[4].split(",")


    Acc_to_Lineage[Accession] = taxID_lineage
    Acc_to_Rank[Accession] = Rank_lineage
      
### IBM
for Line in IBM_Lines[1:]:
    Accession = Line[0]

    taxID = Line[1]

    taxID_lineage = Line[2].split(",")
    Rank_lineage = Line[4].split(",")


    Acc_to_Lineage[Accession] = taxID_lineage
    Acc_to_Rank[Accession] = Rank_lineage

print(len(Acc_to_Lineage))

425473


In [5]:
GB_Lines[2]

['GCA_000007365.1',
 '198804',
 '198804,98794,9,32199,1903409,91347,1236,1224,2,131567',
 'no rank',
 'no rank,no rank,species,genus,family,order,class,phylum,superkingdom,no rank',
 'Buchnera aphidicola str. Sg (Schizaphis graminum)',
 'Buchnera aphidicola str. Sg (Schizaphis graminum),Buchnera aphidicola (Schizaphis graminum),Buchnera aphidicola,Buchnera,Erwiniaceae,Enterobacterales,Gammaproteobacteria,Proteobacteria,Bacteria,cellular organisms\n']

In [6]:
df = pd.read_table("Data/EC_KP_matrix.txt.gz")

In [7]:
df = df.drop("#query", axis=1)

In [8]:
# df.loc[[0,1,2], :]

In [9]:
Accessions = []
for row in list(df):
    entry = row.split("/")[-1]
    if entry.startswith("GCA"):
        entry = entry.split(".")[0]
    if entry.startswith("GCF"):         
            entry = entry.split(".")[0]
            entry = entry.replace("GCF", "GCA")
    if "-" in entry:
        entry = entry.split("-")[0]
    Accessions.append(entry)
    
print(len(list(df)))    
print(len(Accessions))

51006
51006


In [10]:
Lineages_dict = {}
Rank_dict = {}
for Entry in set(Accessions):
    try:
        Lineages_dict[Entry] = Acc_to_Lineage[Entry]
        Rank_dict[Entry] = Acc_to_Rank[Entry]
    except:
        Lineages_dict[Entry] = "NA"
        Rank_dict[Entry] = "NA"
        print(Entry)

print(len(set(Accessions)))
print(len(Lineages_dict))
print(len(Rank_dict))
Rank_dict['GCA_002192315']

ERR1203940
SRR3410227
SRR3410234
SRR3210581
ERR1204039
SRR3951697
SRR1015316
SRR4101538
SRR4101487
SRR3951658
SRR3091368
SRR1871148
SRR4100182
SRR4098814
SRR2014837
SRR3951666
SRR3951686
SRR3214107
SRR3987476
SRR4101501
SRR3951627
SRR3951682
SRR3465427
SRR3986238
SRR4098833
SRR3713558
SRR4098834
SRR3951653
SRR1015320
SRR4098804
SRR3091371
SRR5121090
SRR3458009
SRR3977058
SRR3971181
SRR4101511
SRR3951657
SRR4101488
SRR3951696
SRR1609087
SRR3986260
SRR3713560
SRR3465441
ERR1203966
SRR4100171
SRR3465422
SRR3210595
SRR3210646
SRR1015296
SRR3986252
SRR3951654
SRR4101500
ERR1204036
SRR4100180
SRR3951624
SRR4101540
SRR3951678
ERR1204071
SRR3458008
SRR4101539
SRR1964552
SRR4101529
SRR4099210
SRR3976910
SRR4101513
ERR1203956
SRR3210580
SRR3987468
SRR3466144
SRR3951651
SRR4098820
SRR2449353
SRR4046338
SRR3210641
SRR3987457
SRR3987385
ERR1204050
SRR3951634
SRR3987436
ERR1203945
SRR4101484
SRR3458010
SRR3951661
ERR1204040
SRR3986249
SRR4101491
SRR4099202
SRR3091370
SRR4101528
DRR022926
SRR4101477


['species',
 'genus',
 'family',
 'order',
 'class',
 'phylum',
 'superkingdom',
 'no rank']

In [11]:
Sub_positions = []
Sub_accessions = []
for Acc in Accessions:
    Lineage = Lineages_dict[Acc]
    if  '561' in Lineage or '620' in Lineage or '28901' in Lineage or "573" in Lineage:
        Sub_positions.append(int(Accessions.index(Acc)))
        Sub_accessions.append(Acc)
print(len(Sub_positions))

49430


In [12]:
Sub_species = []
for Acc in Sub_accessions:
    try:
        Lineage = Lineages_dict[Acc]
        Ranks_Lineage = Rank_dict[Acc]
        Index = Ranks_Lineage.index("species")
        Sub_species.append(taxID_to_SciName[Lineage[Index]])
    except:
        print(Lineage, Ranks_Lineage, Index)
        Sub_species.append("NA")

print(len(Sub_species))

['561', '543', '91347', '1236', '1224', '2', '131567'] ['genus', 'family', 'order', 'class', 'phylum', 'superkingdom', 'no rank'] 0
['561', '543', '91347', '1236', '1224', '2', '131567'] ['genus', 'family', 'order', 'class', 'phylum', 'superkingdom', 'no rank'] 0
['561', '543', '91347', '1236', '1224', '2', '131567'] ['genus', 'family', 'order', 'class', 'phylum', 'superkingdom', 'no rank'] 0
['561', '543', '91347', '1236', '1224', '2', '131567'] ['genus', 'family', 'order', 'class', 'phylum', 'superkingdom', 'no rank'] 0
['561', '543', '91347', '1236', '1224', '2', '131567'] ['genus', 'family', 'order', 'class', 'phylum', 'superkingdom', 'no rank'] 0
['561', '543', '91347', '1236', '1224', '2', '131567'] ['genus', 'family', 'order', 'class', 'phylum', 'superkingdom', 'no rank'] 0
['561', '543', '91347', '1236', '1224', '2', '131567'] ['genus', 'family', 'order', 'class', 'phylum', 'superkingdom', 'no rank'] 0
['561', '543', '91347', '1236', '1224', '2', '131567'] ['genus', 'family', '

In [13]:
Sub_genus = []
for Acc in Sub_accessions:
    try:
        Lineage = Lineages_dict[Acc]
        Ranks_Lineage = Rank_dict[Acc]
        Index = Ranks_Lineage.index("genus")
        Sub_genus.append(taxID_to_SciName[Lineage[Index]])
    except:
        print(Lineage, Ranks_Lineage, Index)
        Sub_genus.append("NA")
print(len(Sub_genus))

49430


In [14]:
Submatrix = df.iloc[Sub_positions, Sub_positions]
# Submatrix.head()

In [15]:
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import pylab
import matplotlib.colors as colors
from matplotlib import cm
import numpy as np
from adjustText import adjust_text
from sklearn import manifold 

In [ ]:
mds = manifold.MDS(n_components=2, dissimilarity="precomputed", random_state=2)
results = mds.fit(Submatrix)

In [ ]:
### Create plot
coords = results.embedding_
Plot = pd.DataFrame(dict(x=coords[:, 0], y=coords[:, 1], label=Sub_accessions))
Plot["Species"] = Sub_species
Plot["Genus"] = Sub_genus
Plot.head()

In [ ]:
### Plot parameters
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 12
fig_size[1] = 9
plt.rcParams["figure.figsize"] = fig_size

### Plot
groups = Plot.groupby('Species')
fig, ax = plt.subplots()
ax.margins(0.2) # Optional, just adds 5% padding to the autoscaling

colors = cm.tab10(np.linspace(0, 1, len(groups)))
alphas = np.linspace(.2, .8, len(groups))
for group, color, alpha in zip(groups, colors, alphas):
    ax.plot(group[1].x, 
            group[1].y, 
            marker='o', 
            linestyle='', 
            ms=10, 
            label=group[0], 
            c=color, 
            markeredgecolor='k', 
            alpha=(float(1)/float(len(group[1])**(.2)))
           )

lgd = pylab.legend()
# ax.legend_.remove()
plt.show()

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler
import numpy as np

In [ ]:
X = Plot[['x','y']].values
labels_true = Plot["label"]

In [ ]:
db = DBSCAN(eps=0.05, min_samples=30).fit(X)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)

In [ ]:
print('Estimated number of clusters: %d' % n_clusters_)
print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))


# #############################################################################
# Plot result
import matplotlib.pyplot as plt

# Black removed and is used for noise instead.
unique_labels = set(labels)
colors = [plt.cm.viridis(each)
          for each in np.linspace(0, 1, len(unique_labels))]
for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = [0,0,0, 0]

    class_member_mask = (labels == k)

    xy = X[class_member_mask & core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=14, alpha=.8)

    xy = X[class_member_mask & ~core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=14, alpha=.8)

plt.title('Estimated number of clusters: %d' % n_clusters_)
plt.show()

In [ ]:
print(len(labels))
print(len(labels_true))

In [ ]:
Plot["Cluster"] = labels

In [ ]:
### Plot parameters
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 12
fig_size[1] = 9
plt.rcParams["figure.figsize"] = fig_size

### Plot
groups = Plot.groupby('Cluster')
fig, ax = plt.subplots()
ax.margins(0.2) # Optional, just adds 5% padding to the autoscaling

colors = cm.tab10(np.linspace(0, 1, len(groups)))
alphas = np.linspace(.2, .8, len(groups))
for group, color, alpha in zip(groups, colors, alphas):
    ax.plot(group[1].x, 
            group[1].y, 
            marker='o', 
            linestyle='', 
            ms=10, 
            label=group[0], 
            c=color, 
            markeredgecolor='k', 
            alpha=(float(1)/float(len(group[1])**(.2)))
           )

lgd = pylab.legend()
# ax.legend_.remove()
plt.show()

In [ ]:
Kleb = Plot.loc[Plot['Species'] == 'Klebsiella pneumoniae']
Kleb.head()

In [ ]:
### Plot parameters
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 12
fig_size[1] = 9
plt.rcParams["figure.figsize"] = fig_size

### Plot
groups = Kleb.groupby('Cluster')
fig, ax = plt.subplots()
ax.margins(0.2) # Optional, just adds 5% padding to the autoscaling

colors = cm.tab10(np.linspace(0, 1, len(groups)))
alphas = np.linspace(.2, .8, len(groups))
for group, color, alpha in zip(groups, colors, alphas):
    ax.plot(group[1].x, 
            group[1].y, 
            marker='o', 
            linestyle='', 
            ms=10, 
            label=group[0], 
            c=color, 
            markeredgecolor='k', 
            alpha=(float(1)/float(len(group[1])**(.2)))
           )

lgd = pylab.legend()
# ax.legend_.remove()
plt.show()

In [ ]:
Sub_positions = []
Sub_accessions = []
for Acc in Accessions:
    Lineage = Lineages_dict[Acc]
    if "570" in Lineage:
        Sub_positions.append(int(Accessions.index(Acc)))
        Sub_accessions.append(Acc)
print(len(Sub_positions))

Kleb_Submatrix = df.iloc[Sub_positions, Sub_positions]
Kleb_Submatrix.head()

In [ ]:
mds = manifold.MDS(n_components=2, dissimilarity="precomputed", random_state=2)
results = mds.fit(Submatrix)

In [ ]:
### Create plot
coords = results.embedding_
Plot_kleb = pd.DataFrame(dict(x=coords[:, 0], y=coords[:, 1], label=Sub_accessions))

Plot_kleb.head()

In [ ]:
Kleb.head()

In [ ]:
df = pd.merge(Kleb[['label', 'Cluster']], Plot_kleb, on=['label','label'])
df.head()

In [ ]:
### Plot parameters
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 12
fig_size[1] = 9
plt.rcParams["figure.figsize"] = fig_size

### Plot
groups = df.groupby('Cluster')
fig, ax = plt.subplots()
ax.margins(0.2) # Optional, just adds 5% padding to the autoscaling

colors = cm.viridis(np.linspace(0, 1, len(groups)))
alphas = np.linspace(.2, .8, len(groups))
for group, color, alpha in zip(groups, colors, alphas):
    ax.plot(group[1].x, 
            group[1].y, 
            marker='o', 
            linestyle='', 
            ms=10, 
            label=group[0], 
            c=color, 
            markeredgecolor='k', 
            alpha=(float(1)/float(len(group[1])**(.2)))
           )

lgd = pylab.legend()
ax.legend_.remove()
plt.show()

In [ ]:
from scipy.cluster.hierarchy import linkage
import scipy.spatial.distance as ssd
from scipy.spatial.distance import pdist

In [ ]:
### Create a hierarchical tree ###
distArray = ssd.squareform(Kleb_Submatrix)
row_clusters = linkage(distArray, metric='euclidean', method='complete')

In [ ]:
Kleb_Submatrix.head()

In [ ]:
Sub_species = []
for Acc in Sub_accessions:
    try:
        Lineage = Lineages_dict[Acc]
        Ranks_Lineage = Rank_dict[Acc]
        Index = Ranks_Lineage.index("species")
        Sub_species.append(taxID_to_SciName[Lineage[Index]])
    except:
        print(Lineage, Ranks_Lineage, Index)
        Sub_species.append("NA")

print(len(Sub_species))

In [ ]:
Clustering = pd.DataFrame(row_clusters,
             columns=['row label 1',
                      'row label 2',
                      'distance', 
                      'no. items in cluster'
                     ],
             index=['cluster {}'.format(i+1) for i in range(row_clusters.shape[0])]
            )

In [ ]:
from scipy.cluster.hierarchy import dendrogram
from matplotlib import pyplot as plt

fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 12
fig_size[1] = 9
plt.rcParams["figure.figsize"] = fig_size

row_dendr = dendrogram(row_clusters, labels=Sub_species)

plt.tight_layout()
plt.ylabel('Distance')
plt.grid(False)
plt.show()
# plt.close()

In [ ]:
from scipy.spatial.distance import pdist, squareform
from sklearn import datasets
from scipy.cluster.hierarchy import linkage
import numpy as np
from matplotlib import pyplot
import matplotlib as mpl
from matplotlib.pyplot import cm
from scipy.cluster import hierarchy

In [ ]:
def seriation(Z,N,cur_index):
    '''
        input:
            - Z is a hierarchical tree (dendrogram)
            - N is the number of points given to the clustering process
            - cur_index is the position in the tree for the recursive traversal
        output:
            - order implied by the hierarchical tree Z
            
        seriation computes the order implied by a hierarchical tree (dendrogram)
    '''
    if cur_index < N:
        return [cur_index]
    else:
        left = int(Z[cur_index-N,0])
        right = int(Z[cur_index-N,1])
        return (seriation(Z,N,left) + seriation(Z,N,right))
    
def compute_serial_matrix(dist_mat,method="ward"):
    '''
        input:
            - dist_mat is a distance matrix
            - method = ["ward","single","average","complete"]
        output:
            - seriated_dist is the input dist_mat,
              but with re-ordered rows and columns
              according to the seriation, i.e. the
              order implied by the hierarchical tree
            - res_order is the order implied by
              the hierarhical tree
            - res_linkage is the hierarhical tree (dendrogram)
        
        compute_serial_matrix transforms a distance matrix into 
        a sorted distance matrix according to the order implied 
        by the hierarchical tree (dendrogram)
    '''
    N = len(dist_mat)
    flat_dist_mat = squareform(dist_mat)
    res_linkage = linkage(flat_dist_mat, method=method)
    res_order = seriation(res_linkage, N, N + N-2)
    seriated_dist = np.zeros((N,N))
    a,b = np.triu_indices(N,k=1)
    seriated_dist[a,b] = dist_mat[ [res_order[i] for i in a], [res_order[j] for j in b]]
    seriated_dist[b,a] = seriated_dist[a,b]
    
    return seriated_dist, res_order, res_linkage

In [ ]:
methods = ["ward","single","average","complete"]
for method in methods:
    print("Method:\t",method)
    
    ordered_dist_mat, res_order, res_linkage = compute_serial_matrix(Kleb_Submatrix.values,method)
    
    plt.pcolormesh(ordered_dist_mat, cmap='viridis_r')
    plt.xlim([0,len(ordered_dist_mat)])
    plt.ylim([0,len(ordered_dist_mat)])
    plt.show()
#     plt.close()

In [ ]:
### Combine Tree and Heatmap using ward linkage ###

ordered_dist_mat, res_order, res_linkage = compute_serial_matrix(Kleb_Submatrix.values,'single')
ordered_dist_mat = ordered_dist_mat[::-1]

### Make Plot ###
fig=plt.figure(facecolor='white')
fig.set_size_inches(30, 20)
fig.set_dpi(80)
### Plot Dendrogram ###
axd = fig.add_axes([0.05, 0.1, 0.25, 0.6])

hierarchy.set_link_color_palette(['#42bb72ff', "#0c2a50ff", "#0c2a50ff", "#0c2a50ff"])

row_dendr = dendrogram(res_linkage, 
                       labels=Sub_accessions, 
                       orientation='left', 
                       color_threshold=0.5, 
                       above_threshold_color='#440154ff')

axd.set_xlabel('Jaccard Distance', fontsize=20, labelpad=20)
axd.tick_params(axis='both', which='major', labelsize=20)
axd.axes.get_yaxis().set_visible(False)


### Plot Heatmap ###
axm = fig.add_axes([0.15, 0.1, 0.6, 0.6])

cax = axm.matshow(ordered_dist_mat, interpolation='nearest', cmap='viridis_r')
axm.grid(False)
axm.tick_params(axis='both', which='major', labelsize=20)
axm.axes.get_yaxis().set_visible(False)
axm.xaxis.tick_bottom()
axm.set_xlabel('Genome number', fontsize=20, labelpad=20)

### Color legend ###
colorbar = fig.colorbar(cax,fraction=0.025)
colorbar.ax.invert_yaxis()
colorbar.set_label(label='Jaccard Distance',size=20, labelpad=20)
colorbar.ax.tick_params(labelsize=20) 

### Figure size and resolution ###

# fig.savefig('149539_CCSC_outgroup_subtracted_Hierarchical_tree_and_heatmap_ward-dist.pdf')